In [1]:
# Imports needed for analysis

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

from geopy.distance import geodesic
import geopandas as gpd
from IPython.display import IFrame

# Increasing the complexity of the DataSet by adding extra Features
Book # 3

**The data is divided into 3 datasets with approximately the same number of properties.**

- Declining Resale Market : 1998 (56010 homes sold)
- Increasing Resale Market : 2010 (36985 homes sold)
- Constant Resale Market : 2017 + (45267 homes sold)

**For each dataframe, the following will be added according to the year:**

- Region (North, East, West, Central)
- Additional HDB information (dataset : -------_)
- Nearest MRT
- Nearest Primary School

In [2]:
# importing dataframes

df_low = pd.read_csv('./declining_sale_price_df.csv')
df_high = pd.read_csv('./increasing_sale_price_df.csv')
df_const = pd.read_csv('./constant_sale_price_df.csv')


In [3]:
# reset indexes
df_low.drop(columns = 'Unnamed: 0', inplace = True)
df_high.drop(columns = 'Unnamed: 0', inplace = True)
df_const.drop(columns = 'Unnamed: 0', inplace = True)

### Part 1 - Add the General Region

In [4]:
# Add a Column for the General Region in the dataframe

def region_allocation(df):
    df['region'] = 'empty'
    
    for i in range(len(df)):
    
        for north in ['SEMBAWANG', 'PUNGGOL', 'YISHUN', 'WOODLANDS', 'ANG MO KIO', 'HOUGANG',  'SENGKANG', 'SERANGOON', 'LIM CHU KANG']:
            if df.loc[i,'town'] == north: 
                df.loc[i,'region'] = 'North'
            
        for east in ['BEDOK', 'TAMPINES', 'PASIR RIS']:
            if df.loc[i,'town'] == east: 
                df.loc[i,'region'] = 'East'
            
        for west in ['JURONG WEST', 'BUKIT BATOK','CHOA CHU KANG', 'CLEMENTI', 'BUKIT PANJANG','JURONG EAST']:
            if df.loc[i,'town'] == west: 
                df.loc[i,'region'] = 'West'
            
        for central in ['BISHAN','BUKIT MERAH', 'CENTRAL AREA', 'TOA PAYOH', 'QUEENSTOWN', 'KALLANG/WHAMPOA', 'GEYLANG','MARINE PARADE', 'BUKIT TIMAH']:
            if df.loc[i,'town'] == central: 
                df.loc[i,'region'] = 'Central'   
            

region_allocation(df_low)
region_allocation(df_high)
region_allocation(df_const)

In [5]:
df_const['region'].value_counts()

North      19115
West       10684
Central     8867
East        6601
Name: region, dtype: int64

In [6]:
# save to csv
df_low.to_csv('./df_low_p1.csv')
df_high.to_csv('./df_high_p1.csv')
df_const.to_csv('./df_const_p1.csv')

In [7]:
df_low = pd.read_csv('./df_low_p1.csv')
df_high = pd.read_csv('./df_high_p1.csv')
df_const = pd.read_csv('./df_const_p1.csv')

### Part 2 - Additional HDB Information 
**Assumption: HDB information is valid from the day the building was completed**

In [8]:
hdb_info = pd.read_csv('./hdb-property-information.csv')
hdb_info.drop(columns = ['Unnamed: 0'],inplace = True)

In [9]:
hdb_info.head()

,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,total_dwelling_units,1room_sold,...,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental,full_address
0,16,1970,1,1,0,0,0,0,142,0,...,1,2,0,0,0,0,0,0,0,1 BEACH RD Singapore
1,14,1975,1,0,0,1,0,0,206,0,...,0,2,0,0,0,0,0,0,0,1 BEDOK STH AVE 1 Singapore
2,2,2010,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1 CANTONMENT RD Singapore
3,15,1982,1,0,0,0,0,0,102,0,...,10,92,0,0,0,0,0,0,0,1 CHAI CHEE RD Singapore
4,4,1975,1,1,0,0,0,0,55,0,...,0,1,0,0,0,0,0,0,0,1 CHANGI VILLAGE RD Singapore


In [10]:
df_low = df_low.merge(hdb_info, on='full_address')
df_high = df_high.merge(hdb_info, on='full_address')
df_const = df_const.merge(hdb_info, on='full_address')

### Part 3 - Adding Longitude and Latitude to Datasets

In [11]:
latlong = pd.read_csv('lat_long_complete.csv')
latlong.drop(columns = ['Unnamed: 0'],inplace = True)

In [12]:
latlong.head()

,Address,Latitude,Longitude
0,309 ANG MO KIO AVE 1 Singapore,1.36607,103.84484
1,216 ANG MO KIO AVE 1 Singapore,1.36582,103.84145
2,211 ANG MO KIO AVE 3 Singapore,1.36874,103.84098
3,202 ANG MO KIO AVE 3 Singapore,1.36833,103.84415
4,235 ANG MO KIO AVE 3 Singapore,1.36762,103.83709


In [13]:
latlong.columns = ['full_address', 'Latitude', 'Longitude']

In [14]:
df_low = df_low.merge(latlong, on='full_address')
df_high = df_high.merge(latlong, on='full_address')
df_const = df_const.merge(latlong, on='full_address')

### Part 3 - Adding Closest MRT
**Note: Since the data is from previous years, we need to make 3 dataframes for each of the time periods**

In [15]:
# MRT information with geolocations
df_mrt = pd.read_csv('./mrt_geo.csv')

In [16]:
df_mrt.head()

,line,station_name,year_opened,is_interchange,Latitude,Longitude
0,DT,BUKIT PANJANG MRT STATION SINGAPORE,Dec-15,0,1.378626,103.761431
1,DT,CASHEW MRT STATION SINGAPORE,Dec-15,0,1.369976,103.764579
2,DT,HILLVIEW MRT STATION SINGAPORE,Dec-15,0,1.363038,103.767413
3,DT,BEAUTY WORLD MRT STATION SINGAPORE,Dec-15,0,1.341553,103.775747
4,DT,KING ALBERT PARK MRT STATION SINGAPORE,Dec-15,0,1.335688,103.783193


In [19]:
df_mrt['year_opened'] = pd.to_datetime(df_mrt['year_opened'], format ='%b-%y')

In [21]:
early_mrt = df_mrt[df_mrt['year_opened'] <= '1999-01-01']

mid_mrt = df_mrt[df_mrt['year_opened'] <= '2011-01-01']

late_mrt = df_mrt[df_mrt['year_opened'] <= '2017-01-01']

In [29]:
def near_mrt(df,df_mrt):
    df['nearest_mrt'] = 'none'
    df['nearest_distance'] = 0
    for i, row in df.iterrows():
        coords_1 = (row['Latitude'], row['Longitude'])
        nearest_mrt = []
        nearest_distance = 0
        
        for ii, mrt in df_mrt.iterrows():
            coords_2 = (mrt['Latitude'], mrt['Longitude'])
        
            if nearest_mrt == []:
                nearest_mrt = mrt['station_name']
                nearest_distance = geopy.distance.geodesic(coords_1, coords_2).km
            
            elif geopy.distance.geodesic(coords_1, coords_2).km < nearest_distance:
                nearest_mrt = mrt['station_name']
                nearest_distance = geopy.distance.geodesic(coords_1, coords_2).km
            
        df.loc[i,'nearest_mrt'] = nearest_mrt
        df.loc[i,'nearest_distance'] = nearest_distance


In [30]:
near_mrt(df_low, early_mrt)
near_mrt(df_high, mid_mrt)
near_mrt(df_const, late_mrt)

In [35]:
# reset indexes
df_low.drop(columns = 'Unnamed: 0', inplace = True)
df_high.drop(columns = 'Unnamed: 0', inplace = True)
df_const.drop(columns = 'Unnamed: 0', inplace = True)

In [44]:
df_mrt_int = df_mrt[['station_name','is_interchange']]
df_mrt_int.columns = ['nearest_mrt','is_interchange']

In [48]:
df_low = df_low.merge(df_mrt_int, on='nearest_mrt')
df_high = df_high.merge(df_mrt_int, on='nearest_mrt')
df_const = df_const.merge(df_mrt_int, on='nearest_mrt')

In [49]:
# save to csv
df_low.to_csv('./df_low_p3.csv')
df_high.to_csv('./df_high_p3.csv')
df_const.to_csv('./df_const_p3.csv')

In [59]:
df_low = pd.read_csv('./df_low_p3.csv')
df_high = pd.read_csv('./df_high_p3.csv')
df_const = pd.read_csv('./df_const_p3.csv')

### Part 4 - Adding Primary School information

In [60]:
prim_schools = pd.read_csv('./primary_school_info.csv')

In [61]:
prim_schools.drop(columns = 'Unnamed: 0', inplace = True)
prim_schools.head()

,Ranking,School_Name,Tot_Vacancy,perc_balance_vacancy,Latitude,Longitude
0,1,Nanyang Primary School,370,5.1,1.320847,103.807750
1,2,Tao Nan School,318,11.7,1.305285,103.911553
2,3,Ai Tong School,310,6.1,1.360583,103.833020
3,4,Maha Bodhi School,300,9.1,1.328471,103.901299
4,5,Rosyth School,280,6.7,1.372891,103.875002


In [64]:
def near_school(df,df_ps):
    df['nearest_school'] = 'none'
    df['nearest_distance_school'] = 0
    for i, row in df.iterrows():
        coords_1 = (row['Latitude'], row['Longitude'])
        nearest_school = []
        nearest_distance_school = 0
        
        for ii, school in df_ps.iterrows():
            coords_2 = (school['Latitude'], school['Longitude'])
        
            if nearest_school == []:
                nearest_school = school['School_Name']
                nearest_distance_school = geopy.distance.geodesic(coords_1, coords_2).km
            
            elif geopy.distance.geodesic(coords_1, coords_2).km < nearest_distance_school:
                nearest_school = school['School_Name']
                nearest_distance_school = geopy.distance.geodesic(coords_1, coords_2).km
            
        df.loc[i,'nearest_school'] = nearest_school
        df.loc[i,'nearest_distance_school'] = nearest_distance_school


In [65]:
near_school(df_low, prim_schools)
near_school(df_high, prim_schools)
near_school(df_const, prim_schools)

In [70]:
df_low.head()

,Unnamed: 0,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,2room_rental,3room_rental,other_room_rental,Latitude,Longitude,nearest_mrt,nearest_distance,is_interchange,nearest_school,nearest_distance_school
0,0,1998-01-01,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,10-20,45.0,improved,1986,...,3,0,0,1.37475,103.83609,YIO CHU KANG MRT STATION SINGAPORE,1.250193,0,CHIJ St. Nicholas Girls’ School,0.248192
1,1,1998-01-01,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,10-20,45.0,improved,1986,...,3,0,0,1.37475,103.83609,YIO CHU KANG MRT STATION SINGAPORE,1.250193,0,CHIJ St. Nicholas Girls’ School,0.248192
2,2,1998-01-01,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,1-10,45.0,improved,1986,...,3,0,0,1.37475,103.83609,YIO CHU KANG MRT STATION SINGAPORE,1.250193,0,CHIJ St. Nicholas Girls’ School,0.248192
3,3,1998-02-01,ANG MO KIO,3 ROOM,172,ANG MO KIO AVE 4,1-10,69.0,improved,1986,...,3,0,0,1.37475,103.83609,YIO CHU KANG MRT STATION SINGAPORE,1.250193,0,CHIJ St. Nicholas Girls’ School,0.248192
4,4,1998-03-01,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,1-10,45.0,improved,1986,...,3,0,0,1.37475,103.83609,YIO CHU KANG MRT STATION SINGAPORE,1.250193,0,CHIJ St. Nicholas Girls’ School,0.248192


In [72]:
# merge other info from school for each entry
prim_school = prim_schools[['Ranking', 'School_Name', 'Tot_Vacancy', 'perc_balance_vacancy']]
prim_school.columns = ['Ranking', 'nearest_school', 'Tot_Vacancy', 'perc_balance_vacancy']


df_low = df_low.merge(prim_school, on='nearest_school')
df_high = df_high.merge(prim_school, on='nearest_school')
df_const = df_const.merge(prim_school, on='nearest_school')


In [75]:
# reset indexes
df_low.drop(columns = 'Unnamed: 0', inplace = True)
df_high.drop(columns = 'Unnamed: 0', inplace = True)
df_const.drop(columns = 'Unnamed: 0', inplace = True)

In [76]:
# save to csv
df_low.to_csv('./df_low_p4.csv')
df_high.to_csv('./df_high_p4.csv')
df_const.to_csv('./df_const_p4.csv')